In [1]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier
from Standardising import standard
from Recoding_SIC_Codes import Industry_Division 
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import chi2_contingency


# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
ipo_data.I3 = ipo_data.I3.astype(object) # Converting to object for the moment to tidy up summary statistics
# Run Descriptive Statistics on Data File
#print(ipo_data.dtypes)
# Replacing NaN in C2 with 1 
ipo_data['C2']=ipo_data['C2'].fillna(1)


# Removing negative and 0 values for the word/sentence counts as it is beleived that these are errors
# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].mean())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].mean())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].mean())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].mean())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].mean())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].mean())
ipo_data['C3']=ipo_data['C3'].mask(ipo_data['C3'] == 0,ipo_data['C3'].mean())
# Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())


C:\Users\danielle.ezzo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

# Creating our target and control variables
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)

#Creating ratios for the words, sentences and postive/negative counts. Ratios provide insight
calc(ipo_data)

#Removing redundent data
# del ipo_data['C3']
# del ipo_data['C5']
# del ipo_data['C6']
# # del ipo_data['T1']
# # del ipo_data['T2']
# del ipo_data['T3']
# del ipo_data['T4']
# del ipo_data['T5']
# del ipo_data['S1']
# del ipo_data['S2']
# del ipo_data['S3']
# del ipo_data['P(1Day)']
# del ipo_data['P(L)']
# del ipo_data['P(H)']
# del ipo_data['P(IPO)']
#Looking at the new description of the data
ipo_data.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,...,Y2,C3x,C6x,C5x,T3x,T4x,T5x,S1x,S2x,S3x
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,6.820000e+02,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,13.837666,15.481190,13.515045,25.934766,149.728788,0.863636,1.802750,0.007282,4.935776e+07,1.241519e+07,...,0.708211,0.536657,0.052708,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854
std,6.031467,6.604398,5.792641,72.042293,150.328785,0.343426,158.308525,0.032775,1.039156e+08,2.501761e+07,...,0.454919,0.499020,0.107639,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970
min,3.000000,0.000000,3.000000,0.000000,10.000000,0.000000,-786.239000,-0.162352,3.693227e+06,5.250000e+05,...,0.000000,0.000000,0.000000,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125
25%,10.000000,13.000000,11.000000,11.092500,85.000000,1.000000,-0.750000,-0.013352,1.874682e+07,5.004180e+06,...,0.000000,0.000000,0.000000,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570
50%,13.500000,15.481190,13.000000,15.000000,110.000000,1.000000,0.085000,0.007282,2.759165e+07,7.500000e+06,...,1.000000,1.000000,0.000000,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305
75%,17.000000,17.000000,15.000000,21.030001,152.750000,1.000000,0.620000,0.030941,4.938184e+07,1.208721e+07,...,1.000000,1.000000,0.071429,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,1.000000,3864.500000,0.092896,2.138085e+09,4.212336e+08,...,1.000000,1.000000,1.000000,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397


In [3]:
ipo_data['income']=ipo_data['P(IPO)']*ipo_data['C6']
ipo_data['perOffered']=ipo_data['C6']/ipo_data['C5']
ipo_data['P/E']=ipo_data['P(IPO)']/ipo_data['C3']
ipo_data['P/E']=ipo_data['P/E']+1150
ipo_data.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,...,C5x,T3x,T4x,T5x,S1x,S2x,S3x,income,perOffered,P/E
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,6.820000e+02,...,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,6.820000e+02,682.000000,682.000000
mean,13.837666,15.481190,13.515045,25.934766,149.728788,0.863636,1.802750,0.007282,4.935776e+07,1.241519e+07,...,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854,2.265801e+08,0.304365,1159.904779
std,6.031467,6.604398,5.792641,72.042293,150.328785,0.343426,158.308525,0.032775,1.039156e+08,2.501761e+07,...,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970,9.485423e+08,0.221202,157.354155
min,3.000000,0.000000,3.000000,0.000000,10.000000,0.000000,-786.239000,-0.162352,3.693227e+06,5.250000e+05,...,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125,1.706250e+06,0.010021,0.000000
25%,10.000000,13.000000,11.000000,11.092500,85.000000,1.000000,-0.750000,-0.013352,1.874682e+07,5.004180e+06,...,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570,5.810000e+07,0.205547,1139.437996
50%,13.500000,15.481190,13.000000,15.000000,110.000000,1.000000,0.085000,0.007282,2.759165e+07,7.500000e+06,...,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305,9.775000e+07,0.265248,1152.913835
75%,17.000000,17.000000,15.000000,21.030001,152.750000,1.000000,0.620000,0.030941,4.938184e+07,1.208721e+07,...,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883,1.734750e+08,0.350455,1177.990302
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,1.000000,3864.500000,0.092896,2.138085e+09,4.212336e+08,...,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397,1.786400e+10,3.530786,2950.000000


In [4]:
# Converting C6x and C2 to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)
#Filing days was broken into three seperate groups. Under six month, 6-12 months, and over 12 months. 
ipo_data['C6xbin'] = 0
ipo_data.loc[(ipo_data['C6x'] > 0),'C6xbin'] = 1

sqrt_transform=['income','P/E','perOffered','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation

ipo_data['T5xbin'] = 0
# ipo_data.loc[(ipo_data['T5x'] >= 0.080692),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= 0) & (ipo_data['T5x'] < .0474),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= .047401) & (ipo_data['T5x'] < .0519), 'T5xbin'] = 2
ipo_data.loc[(ipo_data['T5x'] >= .051901) & (ipo_data['T5x'] < .05647), 'T5xbin'] = 3
ipo_data.loc[ipo_data[ 'T5x'] >= .0564701,'T5xbin'] = 4

ipo_data['T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] < 0.897133),'T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] >= 0.897133),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= 0.703925) & (ipo_data['T3x'] < 0.883578),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= .883578) & (ipo_data['T3x'] < 0.897133), 'T3xbin'] = 2
ipo_data.loc[(ipo_data['T3x'] >= .897133) & (ipo_data['T3x'] < 0.909732), 'T3xbin'] = 3
ipo_data.loc[ipo_data[ 'T3x'] >= 0.909732,'T3xbin'] = 4

print(ipo_data.dtypes) # checking the types
print(ipo_data.T5xbin.value_counts())

I1             object
I2             object
I3             object
P(IPO)        float64
P(H)          float64
P(L)          float64
P(1Day)       float64
C1            float64
C2              int32
C3            float64
C4            float64
C5            float64
C6            float64
C7            float64
T1            float64
T2            float64
T3            float64
T4            float64
T5            float64
S1            float64
S2            float64
S3            float64
Y1              int64
Y2              int64
C3x             int64
C6x           float64
C5x           float64
T3x           float64
T4x           float64
T5x           float64
S1x           float64
S2x           float64
S3x           float64
income        float64
perOffered    float64
P/E           float64
C6xbin          int64
T5xbin          int64
T3xbin          int64
dtype: object
4    171
2    171
3    170
1    170
Name: T5xbin, dtype: int64


In [5]:
"""Code to recode the column I3 into industry level.

    Version Control:
    Initial coding
    ------------------------------------------
    Date 8-Feb-18, Author: Danielle Ezzo, Desc: Initial Coding
    """

def Industry_Division1(row,method):
    # Recoding based on table in milestone report
    if method==1:
        if (row['I3']> 99 and row['I3']< 1000):
            return 'Agriculture, Forestry and Fishing'
        elif (row['I3']> 999 and row['I3']< 1500):
            return  'Mining'
        elif (row['I3']> 1499 and row['I3']< 1800):
            return 'Construction'
        elif (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Transportation, Communications, Electric, Gas and Sanitary service'
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Wholesale Trade'
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Retail Trade'
        elif (row['I3']> 5999 and row['I3']< 6800):
            return 'Finance, Insurance and Real Estate'
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        elif (row['I3']> 9099 and row['I3']< 9730):
            return 'Public Administration'
        elif (row['I3']> 9899 and row['I3']< 10000):
            return 'Nonclassifiable'
        else:
            return 'Missing'
    elif method==2:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        else:
            return 'Other'
    elif method==3:
        if (row['I3']> 7369 and row['I3']< 7375) :
            return 'Tech'
        else:
            return 'NonTech'
    elif method==4:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing' # Manufacturing
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' # Tech Comms etc Services
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services' #Services (combining with above)
        else:
            return 'Other' # All Other
    elif method==5:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' #TCEGS
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==6:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'TCEGS' 
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==7:
        if (row['I3']> 99 and row['I3']< 5000):
            return 'Manual Labor'
        else: 
            return 'NonManual Labor'

# Add the new data to the end of the table
ipo_data.I3 = ipo_data.I3.astype(int) # recoding to correct type

# import functools
# import operator


# applying our function to recode our industry codes to sector level
ipo_data['IndDivision'] = ipo_data.apply(Industry_Division1,method=4, axis=1) 
print(ipo_data.head(5))

     I1                           I2    I3  P(IPO)  P(H)  P(L)    P(1Day)  \
0    RA         RAILAMERICA INC /DE   4011    15.0  18.0  16.0  13.750000   
1     G                 Genpact LTD   8742    14.0  18.0  16.0  16.750000   
2  HURN  Huron Consulting Group Inc.  8742    15.0  16.0  14.0  18.900000   
3  ICFI     ICF International, Inc.   8742    12.0  16.0  14.0  12.250000   
4  SRVY       GREENFIELD ONLINE INC   8742    13.0  15.0  13.0  18.700001   

         C1  C2    C3     ...            S1x       S2x       S3x  \
0  4.343805   1  0.30     ...       0.082570  0.122955  0.081668   
1  4.418841   1  0.19     ...       0.077702  0.086114  0.109887   
2  5.030438   1 -0.14     ...       0.055644  0.070385  0.090297   
3  4.941642   1  0.22     ...       0.075398  0.081162  0.126022   
4  4.553877   1  0.07     ...       0.081974  0.095998  0.109543   

         income  perOffered        P/E  C6xbin  T5xbin  T3xbin  IndDivision  
0  18165.902125    0.636244  34.641016       0    

In [6]:
dummies = pd.get_dummies(ipo_data['IndDivision'])

ipo_data = ipo_data.join(dummies)

In [7]:
ipo_data.C6xbin.value_counts()

0    427
1    255
Name: C6xbin, dtype: int64

In [8]:
ipo_data['Manufacturing'] = ipo_data.apply(Industry_Division1,method=2, axis=1) 
ipo_data['Tech'] = ipo_data.apply(Industry_Division1,method=3, axis=1)
ipo_data['IndCluster3'] = ipo_data.apply(Industry_Division1,method=4, axis=1)
ipo_data['IndCluster4'] = ipo_data.apply(Industry_Division1,method=5, axis=1)
ipo_data['IndCluster5'] = ipo_data.apply(Industry_Division1,method=6, axis=1)
ipo_data['Labor'] = ipo_data.apply(Industry_Division1,method=7, axis=1)
print(ipo_data.head(5))

     I1                           I2    I3  P(IPO)  P(H)  P(L)    P(1Day)  \
0    RA         RAILAMERICA INC /DE   4011    15.0  18.0  16.0  13.750000   
1     G                 Genpact LTD   8742    14.0  18.0  16.0  16.750000   
2  HURN  Huron Consulting Group Inc.  8742    15.0  16.0  14.0  18.900000   
3  ICFI     ICF International, Inc.   8742    12.0  16.0  14.0  12.250000   
4  SRVY       GREENFIELD ONLINE INC   8742    13.0  15.0  13.0  18.700001   

         C1  C2    C3       ...         T3xbin  IndDivision  Manufacturing  \
0  4.343805   1  0.30       ...              1     Services          Other   
1  4.418841   1  0.19       ...              3     Services          Other   
2  5.030438   1 -0.14       ...              1     Services          Other   
3  4.941642   1  0.22       ...              3     Services          Other   
4  4.553877   1  0.07       ...              3     Services          Other   

   Other  Services     Tech  IndCluster3  IndCluster4  IndCluster5  

# Spearman

In [10]:
corr=['C1','C4','C5x','C6x','C7','T1','T2','T3x','T4x','T5x','S1x','S2x','S3x','income','perOffered','P/E','Y1','Y2']
ipo_data[corr].corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

# ANOVA

In [11]:
#set up Tech/non tech groups
Tech=ipo_data.query('Tech == "Tech"')
NonTech=ipo_data.query('Tech == "NonTech"')

In [13]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    T=Tech[i]
    N=NonTech[i]
        #run anova
    anova = stats.f_oneway(T, N)
    print('Tech Cluster v Variable {} Result {}'.format(i,anova))

Tech Cluster v Variable C3x Result F_onewayResult(statistic=0.13348488986091356, pvalue=0.714959413649914)
Tech Cluster v Variable C5x Result F_onewayResult(statistic=0.34267820673425764, pvalue=0.55848158272098791)
Tech Cluster v Variable C1 Result F_onewayResult(statistic=1.7971595485676146, pvalue=0.18050395358977259)
Tech Cluster v Variable T1 Result F_onewayResult(statistic=7.9412445013329656, pvalue=0.004972440380415848)
Tech Cluster v Variable T2 Result F_onewayResult(statistic=10.58715961676479, pvalue=0.0011951186638340448)
Tech Cluster v Variable T4x Result F_onewayResult(statistic=5.3726606104292323, pvalue=0.020750622010505045)
Tech Cluster v Variable S1x Result F_onewayResult(statistic=4.2796650763635604, pvalue=0.038948327809137463)
Tech Cluster v Variable S2x Result F_onewayResult(statistic=8.3180511415386675, pvalue=0.0040494285709776348)
Tech Cluster v Variable S3x Result F_onewayResult(statistic=1.1921562748697427, pvalue=0.27528173565854519)
Tech Cluster v Variable C

In [14]:
#set up Labor/non Labor tech groups
Labor=ipo_data.query('Labor == "Manual Labor"')
NonLabor=ipo_data.query('Labor ==  "NonManual Labor"')

In [15]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    T=Labor[i]
    N=NonLabor[i]
        #run anova
    anova = stats.f_oneway(T, N)
    print('Labor Cluster v Variable {} Result {}'.format(i,anova))

Labor Cluster v Variable C3x Result F_onewayResult(statistic=16.751846396850258, pvalue=4.7717664612275293e-05)
Labor Cluster v Variable C5x Result F_onewayResult(statistic=4.8107832143462312, pvalue=0.02862003995938452)
Labor Cluster v Variable C1 Result F_onewayResult(statistic=3.2068766273915346, pvalue=0.073774029777481973)
Labor Cluster v Variable T1 Result F_onewayResult(statistic=1.8683493807557585, pvalue=0.1721159628484549)
Labor Cluster v Variable T2 Result F_onewayResult(statistic=2.7560234104236487, pvalue=0.097349827650420037)
Labor Cluster v Variable T4x Result F_onewayResult(statistic=11.243564956429383, pvalue=0.00084329482432906266)
Labor Cluster v Variable S1x Result F_onewayResult(statistic=1.8263049150271873, pvalue=0.17701401944373857)
Labor Cluster v Variable S2x Result F_onewayResult(statistic=8.7583081934553277, pvalue=0.0031890992746831403)
Labor Cluster v Variable S3x Result F_onewayResult(statistic=1.6575259221230985, pvalue=0.19837570895096565)
Labor Cluster

*Labor Correlated with  P/E (0.7276)

In [16]:
#set up Labor/non Labor tech groups
Manufacturing=ipo_data.query('Manufacturing == "Manufacturing"')
Other=ipo_data.query('Manufacturing ==  "Other"')

In [17]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    T=Manufacturing[i]
    N=Other[i]
        #run anova
    anova = stats.f_oneway(T, N)
    print('Manufacturing Cluster v Variable {} Result {}'.format(i,anova))

Manufacturing Cluster v Variable C3x Result F_onewayResult(statistic=36.684653181458195, pvalue=2.2978831972587645e-09)
Manufacturing Cluster v Variable C5x Result F_onewayResult(statistic=0.30383849704152582, pvalue=0.58166703991672808)
Manufacturing Cluster v Variable C1 Result F_onewayResult(statistic=2.2504553246454071, pvalue=0.13403922778361038)
Manufacturing Cluster v Variable T1 Result F_onewayResult(statistic=8.0740954774610056, pvalue=0.0046247145231639031)
Manufacturing Cluster v Variable T2 Result F_onewayResult(statistic=5.6865898900590572, pvalue=0.017368153739536072)
Manufacturing Cluster v Variable T4x Result F_onewayResult(statistic=5.0271862060439476, pvalue=0.025273980883969194)
Manufacturing Cluster v Variable S1x Result F_onewayResult(statistic=9.9219806260204724, pvalue=0.0017047375907115712)
Manufacturing Cluster v Variable S2x Result F_onewayResult(statistic=2.187176698621411, pvalue=0.1396272667822821)
Manufacturing Cluster v Variable S3x Result F_onewayResult(

*Manufacturing Correlated with  P/E (0.6033), C4 (0.9640)*

In [18]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.IndCluster3.values)
    d_data = {grp:ipo_data[i][ipo_data.IndCluster3 == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data['Manufacturing'], d_data['Services'], d_data['Other'])
    print('IndCluster3 v Variable {} Result {}'.format(i,anova))

IndCluster3 v Variable C3x Result F_onewayResult(statistic=22.522009398492585, pvalue=3.3851737607512834e-10)
IndCluster3 v Variable C5x Result F_onewayResult(statistic=2.8660763907644409, pvalue=0.057610740166433624)
IndCluster3 v Variable C1 Result F_onewayResult(statistic=1.2291287928227876, pvalue=0.29319740074467876)
IndCluster3 v Variable T1 Result F_onewayResult(statistic=8.2804555437350196, pvalue=0.00027989745106686786)
IndCluster3 v Variable T2 Result F_onewayResult(statistic=7.2650395609353167, pvalue=0.00075529823117885346)
IndCluster3 v Variable T4x Result F_onewayResult(statistic=2.5153710661572877, pvalue=0.081585919393551767)
IndCluster3 v Variable S1x Result F_onewayResult(statistic=12.472704638909411, pvalue=4.7896557235672212e-06)
IndCluster3 v Variable S2x Result F_onewayResult(statistic=6.6481303142902588, pvalue=0.0013824810371474232)
IndCluster3 v Variable S3x Result F_onewayResult(statistic=7.8808579886040295, pvalue=0.00041353035727346348)
IndCluster3 v Variabl

*IndCluster3 Correlated with  P/E (0.7547), C4 (0.9099)*

In [19]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.IndCluster4.values)
    d_data = {grp:ipo_data[i][ipo_data.IndCluster4 == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data['Manufacturing'], d_data['Services'], d_data['Trade'], d_data['Other'])
    print('IndCluster4 v Variable {} Result {}'.format(i,anova))

IndCluster4 v Variable C3x Result F_onewayResult(statistic=15.285162319765051, pvalue=1.238625005007192e-09)
IndCluster4 v Variable C5x Result F_onewayResult(statistic=1.9233352039889715, pvalue=0.12443693658428352)
IndCluster4 v Variable C1 Result F_onewayResult(statistic=2.328609893182759, pvalue=0.073339382818282561)
IndCluster4 v Variable T1 Result F_onewayResult(statistic=5.8550885869822631, pvalue=0.00059751555285465839)
IndCluster4 v Variable T2 Result F_onewayResult(statistic=4.9098886707136726, pvalue=0.0022103618841530469)
IndCluster4 v Variable T4x Result F_onewayResult(statistic=1.987039255973593, pvalue=0.11459427873497095)
IndCluster4 v Variable S1x Result F_onewayResult(statistic=8.4468739958451238, pvalue=1.6226803976863386e-05)
IndCluster4 v Variable S2x Result F_onewayResult(statistic=9.4987927499121589, pvalue=3.7492164617250237e-06)
IndCluster4 v Variable S3x Result F_onewayResult(statistic=7.9858539185501032, pvalue=3.0842474164673827e-05)
IndCluster4 v Variable C2

*IndCluster4 Correlated with  C6x (0.5708)*

In [20]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.IndCluster5.values)
    d_data = {grp:ipo_data[i][ipo_data.IndCluster5 == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data['Manufacturing'], d_data['Services'], d_data['TCEGS'], d_data['Trade'], d_data['Other'])
    print('IndCluster5 v Variable {} Result {}'.format(i,anova))

IndCluster5 v Variable C3x Result F_onewayResult(statistic=11.479884452519501, pvalue=5.0643203320909237e-09)
IndCluster5 v Variable C5x Result F_onewayResult(statistic=3.8411273375590249, pvalue=0.0042751900553998244)
IndCluster5 v Variable C1 Result F_onewayResult(statistic=3.333867604500643, pvalue=0.010225871430645541)
IndCluster5 v Variable T1 Result F_onewayResult(statistic=4.6042649636908761, pvalue=0.0011289608923246066)
IndCluster5 v Variable T2 Result F_onewayResult(statistic=3.6794535559786761, pvalue=0.0056526029476907155)
IndCluster5 v Variable T4x Result F_onewayResult(statistic=2.6966013595557801, pvalue=0.029933639552335058)
IndCluster5 v Variable S1x Result F_onewayResult(statistic=6.6871750156401237, pvalue=2.7884073616505167e-05)
IndCluster5 v Variable S2x Result F_onewayResult(statistic=8.2932667952532153, pvalue=1.5598987762446622e-06)
IndCluster5 v Variable S3x Result F_onewayResult(statistic=6.6762300558289018, pvalue=2.8435891807132307e-05)
IndCluster5 v Variabl

*IndCluster4 Correlated with C6x (0.5708)*

In [21]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.T3xbin.values)
    d_data = {grp:ipo_data[i][ipo_data.T3xbin == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data[1], d_data[2], d_data[3], d_data[4])
    print('T3xbin v Variable {} Result {}'.format(i,anova))

T3xbin v Variable C3x Result F_onewayResult(statistic=3.6974254970392857, pvalue=0.011668610635687486)
T3xbin v Variable C5x Result F_onewayResult(statistic=1.1566718217118206, pvalue=0.32549865662124428)
T3xbin v Variable C1 Result F_onewayResult(statistic=1.6570488383517468, pvalue=0.17499844207527826)
T3xbin v Variable T1 Result F_onewayResult(statistic=1.7737376474277171, pvalue=0.15082076389668284)
T3xbin v Variable T2 Result F_onewayResult(statistic=5.7912996252960802, pvalue=0.00065279228016099577)
T3xbin v Variable T4x Result F_onewayResult(statistic=3.2233223560342341, pvalue=0.022202399512796435)
T3xbin v Variable S1x Result F_onewayResult(statistic=7.0209519690892916, pvalue=0.00011821356732629749)
T3xbin v Variable S2x Result F_onewayResult(statistic=2.0649630324003825, pvalue=0.10356708770353737)
T3xbin v Variable S3x Result F_onewayResult(statistic=17.334766558817861, pvalue=7.4480224672478992e-11)
T3xbin v Variable C2 Result F_onewayResult(statistic=1.5102926564456729, p

*T3xbin Correlated with C6x (0.7474), P/E(0.6012)*

In [22]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.T5xbin.values)
    d_data = {grp:ipo_data[i][ipo_data.T5xbin == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data[1], d_data[2], d_data[3], d_data[4])
    print('T5xbin v Variable {} Result {}'.format(i,anova))

T5xbin v Variable C3x Result F_onewayResult(statistic=13.307847567548935, pvalue=1.8942409425887808e-08)
T5xbin v Variable C5x Result F_onewayResult(statistic=1.9336043301036572, pvalue=0.12279739186477739)
T5xbin v Variable C1 Result F_onewayResult(statistic=3.4226178824382321, pvalue=0.016952951985385246)
T5xbin v Variable T1 Result F_onewayResult(statistic=1.7870777395914914, pvalue=0.14826750038857381)
T5xbin v Variable T2 Result F_onewayResult(statistic=1.9091549064628295, pvalue=0.12673534966412534)
T5xbin v Variable T4x Result F_onewayResult(statistic=1.3210449114113771, pvalue=0.2664185622307832)
T5xbin v Variable S1x Result F_onewayResult(statistic=9.7958262540703043, pvalue=2.4794524068946706e-06)
T5xbin v Variable S2x Result F_onewayResult(statistic=5.0084500006630988, pvalue=0.001929173799471005)
T5xbin v Variable S3x Result F_onewayResult(statistic=21.013943899440317, pvalue=5.0098360513451667e-13)
T5xbin v Variable C2 Result F_onewayResult(statistic=3.7344881993853756, pv

*T5xbin Correlated with income (0.9461)*

In [23]:
for i in (['C3x','C5x','C1','T1','T2','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T5x','income','perOffered','P/E']):
    grps = pd.unique(ipo_data.C6xbin.values)
    d_data = {grp:ipo_data[i][ipo_data.C6xbin == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data[0], d_data[1])
    print('C6xbin v Variable {} Result {}'.format(i,anova))

C6xbin v Variable C3x Result F_onewayResult(statistic=12.552505891285346, pvalue=0.00042273471183163793)
C6xbin v Variable C5x Result F_onewayResult(statistic=0.58837301689766197, pvalue=0.44331518316770979)
C6xbin v Variable C1 Result F_onewayResult(statistic=2.7468300729044204, pvalue=0.09790869578058678)
C6xbin v Variable T1 Result F_onewayResult(statistic=0.94244442000702544, pvalue=0.33199516830620612)
C6xbin v Variable T2 Result F_onewayResult(statistic=0.25269902927491467, pvalue=0.61534374729519548)
C6xbin v Variable T4x Result F_onewayResult(statistic=2.1321092139128424, pvalue=0.14470399528684763)
C6xbin v Variable S1x Result F_onewayResult(statistic=3.2630661303773696, pvalue=0.071298504069610774)
C6xbin v Variable S2x Result F_onewayResult(statistic=7.3958884517580001, pvalue=0.0067042025995557846)
C6xbin v Variable S3x Result F_onewayResult(statistic=1.5612602187868483, pvalue=0.21191069608921237)
C6xbin v Variable C2 Result F_onewayResult(statistic=8.7627287154075031, pva

*C6xbin Correlated with C7 (0.6813), T2 (0.6153)*

# Chi Square Tests

In [76]:
for i in (['C2','C6xbin','T3xbin','T5xbin','IndCluster3','IndCluster4','IndCluster5','Tech','Manufacturing','Labor', 'Y1','Y2']):
        for i2 in (['C2','C6xbin','T3xbin','T5xbin', 'Y1','Y2']):
            if i != i2:
                cat_ct = pd.crosstab(ipo_data[i],ipo_data[i2])
                #run chi2 test on crosstab
                chi2, p, dof, expected = chi2_contingency(cat_ct)
                print('{} v {} Chi2 {} p {}'.format(i,i2,chi2,p))
            else:
                'Null'

C2 v C6xbin Chi2 8.010672776444487 p 0.004650245809970422
C2 v T3xbin Chi2 4.527353816257186 p 0.20986332826123275
C2 v T5xbin Chi2 11.08636744941128 p 0.011267904778504208
C2 v Y1 Chi2 2.285998429809462 p 0.13054610936783412
C2 v Y2 Chi2 3.266926328274258 p 0.07068995355610629
C6xbin v C2 Chi2 8.010672776444487 p 0.004650245809970422
C6xbin v T3xbin Chi2 5.1673620516459025 p 0.15994434726821985
C6xbin v T5xbin Chi2 10.322055782020016 p 0.01601787946110845
C6xbin v Y1 Chi2 426.09029482952576 p 1.1530389534756817e-94
C6xbin v Y2 Chi2 39.07713769629338 p 4.07386772568667e-10
T3xbin v C2 Chi2 4.527353816257186 p 0.20986332826123275
T3xbin v C6xbin Chi2 5.1673620516459025 p 0.15994434726821985
T3xbin v T5xbin Chi2 121.5785995868483 p 6.347463942537665e-22
T3xbin v Y1 Chi2 9.264576017174992 p 0.025972379940041434
T3xbin v Y2 Chi2 5.412931512823895 p 0.14394002099253747
T5xbin v C2 Chi2 11.086367449411282 p 0.011267904778504206
T5xbin v C6xbin Chi2 10.322055782020016 p 0.01601787946110845
T5

T5xbin v Y2 0.8408854441319285, Labor v T3xbin 0.580984259015807,

In [8]:
'''
x=pd.DataFrame.copy(ipo_data)
for i in range(len(x.columns)): 
    if x.iloc[:,i].dtype == float:
        outlier(x.iloc[:,i],4)             
for i in range(len(x.columns)):        
    if x.iloc[:,i].dtype == float:   
        x.iloc[:,i]=standard(x.iloc[:,i],1)


# Create K-means model and determine euclidian distances for each data point
#train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other']
train=['perOffered','income','P/E','C3x','C1','T3xbin','T4x','S1x','S2x','S3x','C2','T2','T1','C6xbin','Other','Manufacturing']
X=x[train]
kmeans_model = KMeans(n_clusters=2)
distances = kmeans_model.fit_transform(X)

# Create scatter plot using labels from K-means model as color
labels = kmeans_model.labels_


x['labels'] = labels
train.append('labels')
'''

C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


In [9]:
#print(x.head(5))

     I1                           I2    I3    P(IPO)    P(H)    P(L)  \
0    RA         RAILAMERICA INC /DE   4011  0.500000  0.6875  0.6875   
1     G                 Genpact LTD   8742  0.458333  0.6875  0.6875   
2  HURN  Huron Consulting Group Inc.  8742  0.500000  0.5625  0.5625   
3  ICFI     ICF International, Inc.   8742  0.375000  0.5625  0.5625   
4  SRVY       GREENFIELD ONLINE INC   8742  0.416667  0.5000  0.5000   

    P(1Day)        C1  C2        C3   ...    perOffered       P/E  C6xbin  \
0  0.382795  0.327835   1  0.566929   ...      0.713752  0.768255       0   
1  0.466314  0.360190   1  0.545276   ...      0.290540  0.919586       0   
2  0.526169  0.623910   1  0.480315   ...      0.557648  0.374358       0   
3  0.341036  0.585622   1  0.551181   ...      0.646033  0.797414       0   
4  0.520601  0.418417   1  0.521654   ...      0.551954  0.626279       0   

   T5xbin  T3xbin  IndDivision  Manufacturing  Other  Services  labels  
0       1       1     Services 

In [10]:
#x.to_csv('Model 28_02_18.csv')